# Using other languages

One of the biggest difficulties that appear when changing from another languages to Julia is not being able to keep using all the code that we have developed in other languages. However, Julia allows to invoke code that is written in other languages such as Python, R or C.

## Python

You can import any python package using the *PyCall* package.

In [1]:
using PyCall

┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1278


In [2]:
math = PyCall.pyimport("math")

PyObject <module 'math' (built-in)>

In [3]:
math.sin(math.pi / 4) 

0.7071067811865475

We can also write our own Python code as follows

In [4]:
py""" 
import math

def some_fancy_function(x):
    return math.sin(x)

"""

In [5]:
y = py"some_fancy_function(3)"

0.1411200080598672

If the above function was in a file called *fancy_functions.py*, you can call it as follows:

    python_fancy_functions = pyimport("fancy_functions")
    python_fancy_functions.some_fancy_function(3)

## R code

In [6]:
using RCall

┌ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414]
└ @ Base loading.jl:1278


We can use the *rcall* function

In [7]:
r = rcall(:sum, Float64[1.0, 4.0, 6.0]) 

RObject{RealSxp}
[1] 11


In [8]:
typeof(r)

RObject{RealSxp}

In [9]:
typeof(r[1])

Float64

The @rput allows you to put a Julia variable in the R context.

In [11]:
z = 5
@rput z

5

In [12]:
r = R"z+z"

RObject{IntSxp}
[1] 10


You can apply R functions on Julia variables

In [13]:
x = randn(10) # Julia variable

10-element Array{Float64,1}:
 -0.06138419922610176
  0.8630034882672168
 -0.7189071104322297
  0.34628032100823947
 -1.9944705723399243
 -0.7472448212055829
 -1.946704094008588
 -0.833111618588853
 -0.5903174732956734
 -0.6214720997244313

In [14]:
@rimport base as rbase 
rbase.sum([1,2,3])

RObject{IntSxp}
[1] 6


In [15]:
@rlibrary boot 

In [16]:
R"t.test($x)"

RObject{VecSxp}

	One Sample t-test

data:  `#JL`$x
t = -2.2343, df = 9, p-value = 0.05233
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -1.268734798  0.007869162
sample estimates:
 mean of x 
-0.6304328 



And the equivalent in Julia would be:

In [17]:
using HypothesisTests
OneSampleTTest(x)

┌ Info: Precompiling HypothesisTests [09f84164-cd44-5f33-b23f-e6b0d136a0d5]
└ @ Base loading.jl:1278


One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          -0.6304328179545928
    95% confidence interval: (-1.2687, 0.0079)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.0523

Details:
    number of observations:   10
    t-statistic:              -2.2342686672702596
    degrees of freedom:       9
    empirical standard error: 0.282165178785249


## C and C++ code

Calling standard libraries is easy:

In [18]:
?ccall

search: ccall @ccall AbstractChannel JammalamadakaCircularCorrelation



```
ccall((function_name, library), returntype, (argtype1, ...), argvalue1, ...)
ccall(function_name, returntype, (argtype1, ...), argvalue1, ...)
ccall(function_pointer, returntype, (argtype1, ...), argvalue1, ...)
```

Call a function in a C-exported shared library, specified by the tuple `(function_name, library)`, where each component is either a string or symbol. Instead of specifying a library, one can also use a `function_name` symbol or string, which is resolved in the current process. Alternatively, `ccall` may also be used to call a function pointer `function_pointer`, such as one returned by `dlsym`.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression.

Each `argvalue` to the `ccall` will be converted to the corresponding `argtype`, by automatic insertion of calls to `unsafe_convert(argtype, cconvert(argtype, argvalue))`. (See also the documentation for [`unsafe_convert`](@ref Base.unsafe_convert) and [`cconvert`](@ref Base.cconvert) for further details.) In most cases, this simply results in a call to `convert(argtype, argvalue)`.


In [19]:
t = ccall(:clock, Int32, ())

29006482

You can also wrap your own C code:

In [20]:
"""
#include <stdio.h>
int hello_world()
{
    printf("Hello, World!\n");
    return 0;
}

int hello_world_repeated(int n)
{
    for (int i = 0; i<n; i++){
        printf("Hello, World!\n");
    }
    return 0;
}

To compile:
> gcc -g -O -c c-code/hello_world.c 
> gcc -dynamiclib -o c-code/hello_world_lib.dylib c-code/hello_world.o

""";

In [21]:
ccall(("hello_world","$(pwd())/c-code/hello_world_lib.dylib"),
    Int64,
    (),)

LoadError: TypeError: in ccall: first argument not a pointer or valid constant expression, expected Ptr, got a value of type Tuple{String,String}

In [22]:
ccall((:hello_world_repeated,"$(pwd())/hello_world_lib.dylib"),
    Int64,
    (Int64,),
    10)

LoadError: TypeError: in ccall: first argument not a pointer or valid constant expression, expected Ptr, got a value of type Tuple{Symbol,String}